# Welcome to spinQICK!

## Introduction
In this demo notebook we will show you around some of the basic procedures for setting up your first experiment. First we will set up the communication with the RFSoC, before introducing PyDantic models, which are used to hold pertinent experimental parameters. Next we will discuss the Voltage Source object, before finally showing a basic experiment to explain how the parts come together.

In [ ]:
import Pyro4
from qick import QickConfig
import matplotlib.pyplot as plt
from spinqick.helper_functions.spinqick_enums import GateTypes
from spinqick.models import (
    hardware_config_models,
    full_experiment_model,
    dcs_model,
    qubit_models,
    spam_models,
)
from spinqick.helper_functions import file_manager, hardware_manager
from spinqick.experiments import system_calibrations

## Connecting to the RFSoC: Pyro4 

First we can connect to our qick board.  We use a pyro server to communicate with our board, which allows us to run python code on our computer instead of on the small cpu provided on the board. The package necessary to do this, Pyro4 (and only Pyro4), does not come natively installed with QICK. To learn more about using Pyro4 with QICK, you can find documentation [here](https://github.com/openquantumhardware/qick/blob/main/pyro4/00_nameserver.ipynb). 


If the board is attached to the web than one can install Pyro4 by using `sudo pip3 install Pyro4` or using apt with `sudo apt install python3-pyro4`. If your board is only on a local network, then the Pyro4 .tar files (which can be found [here](https://pypi.org/project/Pyro4/#files)), and serpent (which can be found [here](https://pypi.org/project/serpent/#files)). Check to make sure the versions are compatible (as of this release, Pyro4 4.83 and seprent 1.41). The files can be moved to a local directory using PuTTY/PSCP (a demonstration of how to do this can be found in the QICK quick start guide [here](https://qick-docs.readthedocs.io/latest/quick_start.html#copy-the-qick-tools-onto-your-rfsoc)). 

Once installed, follow the steps found in the [`qick/pyro4`](https://github.com/openquantumhardware/qick/tree/main/pyro4) directory to instantiate the nameserver and local instrument server. You will need to make sure that your `proxy_name`, ip_address and port match those found when instantiating the local instrument server ([`qick/pyro4/01_server.ipynb`](https://github.com/openquantumhardware/qick/blob/main/pyro4/01_server.ipynb)). Once this is complete, you can move to the next cell and begin controlling the RFSoC!


In [ ]:
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION = 4

ns_host = "192.168.2.99"  # <- make sure this matches your board's ip address!
ns_port = 8888  # <- make sure this matches your nameserver's port address!
proxy_name = "myqick"  # <- make sure this matches your instrument's proxy_name!

ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)

## File Settings 

Now that we have confirmed our connection to the RFSoC we can begin setting up our experiment. The first step is to tell spinQICK where to look for the underlying files that define our experimental setup. Later in this demo we will define what those files look like, and how to interact with them, but to begin we want to create a configuration file that just has these locations stored. To do so we will edit the `settings.py` file in the spinqick repository and setting these variables:

    data_directory: str = "C:/data
    hardware_config: str = "C:/data/hardware_cfg.json"
    dot_experiment_config: str = "C:/data/full_config.json"

You'll need to set the `data_directory` variable to a file location on your machine where you can autosave data. The `hardware_config` and `dot_experiment_config` variables specify the location of the respective hardware and experiment json files created in the following steps.

#### spinQICK Config Hierarchy
<p align="center">
<img src="demo_figures/Config_heirarchy.png" alt="Config Heirarchy" width="50%"/>
<figcaption><b>Figure 1. spinQICK Configuration Heirarchy:</b> The above shows the structure of the hardware (left) and experiment (right) configs. These configurations are provided to spinQICK experiments as json files. These configs are intended to be extensible, with the ability to create arbitrary numbers of objects associated with qubits and their properties. The above experiment config is designed primarily for use with EO qubits, however the parameters can also be used for single spin LD qubits as well.</figcaption>
</p>

## Hardware Configs using PyDantic Models 

Dot experiments can be complex and require keeping track and storing a large number of parameters.  Most of the parameters used in spinQICK are passed into the experiment code using a dictionary which we call a _config_. Some parameters in the config are shared across many experiments, for example the setup of readout parameters, like AC source-drain amplitude. We store these parameters in dictionaries which are saved to files (currently in json format), with filepaths specified in `spinqick.settings.filesettings` as shown above. The dictionaries are wrapped as [PyDantic](https://docs.pydantic.dev/latest/) models, which provide schema and data validation to ensure entries are of the correct type, within required value ranges, and item keys are correct. PyDantic also automatically turns keys into object members of the parent model, allowing for dot/autocompletion and linting, reducing the need to interact with the underlying config file or remember key values. PyDantic also natively emits JSON schema.

We can use the hardware and experiment config templates located in `spinqick.models` to create some example configs. We start with the hardware config, which is a hardware map describing all of the low-speed and high-speed lines connecting to each gate. In our example we will define the output generator associated with one of our source-drain inputs of our dot-charge sensor (DCS), and the complementary ADC, with the address associated with the QICK signal generator and ADC channels found in the soccfg above. Using PyDantic models, we can then pass these configs into the primary `hardware_config_models.HardwareConfig`, which represents our simplified hardware setup. We can then save that config into a JSON file. 

Here we will demonstrate how to make the necessary json files to run spinqick experiments!  First, input the location you'd like to save these files below.


In [ ]:
from spinqick.settings import file_settings

hw_file = file_settings.hardware_config
exp_file = file_settings.dot_experiment_config

Now we can begin supplying models for the readout in and out (of which there are two drain lines per 6-dot device), and define fast gates (a---e), and slow gates (f, z1, and z2), before passing these values into the hardware config model and finally saving as a JSON.

In [ ]:
# these parameters define the readout DAC and ADC channels
readout_in = hardware_config_models.SourceDrainIn(
    qick_gen=0, unit_conversion=1, sd_units="dac_units"
)
readout_out_1 = hardware_config_models.SourceDrainOut(
    qick_adc=0, unit_conversion=1, adc_units="adc_units"
)
readout_out_2 = hardware_config_models.SourceDrainOut(
    qick_adc=1, unit_conversion=1, adc_units="adc_units"
)

# FastGate models are used for gates which have both a high speed QICK channel and a low speed DAC channel associated with them
gate_a = hardware_config_models.FastGate(
    dc_conversion_factor=4,  # conversion factor between DC source and gate voltage
    slow_dac_address="slow_dac_address",  # address for DC source hardware
    slow_dac_channel=1,  # DC source channel number
    crosscoupling=None,  # crosscoupling matrix elements, optional
    dac_conversion_factor=1,  # RFSoC units to gate voltage conversion factor
    gate_type=GateTypes.PLUNGER,  # gate type description
    qick_gen=5,  # qick generator number
    max_v=2,  # sets the max DC voltage which can be applied on this channel
)
gate_b = hardware_config_models.FastGate(
    dc_conversion_factor=4,
    slow_dac_address="slow_dac_address",
    dac_conversion_factor=1,
    gate_type=GateTypes.EXCHANGE,
    qick_gen=6,
    slow_dac_channel=2,
    crosscoupling=None,
    max_v=2,
)
gate_c = hardware_config_models.FastGate(
    dc_conversion_factor=4,
    slow_dac_address="slow_dac_address",
    dac_conversion_factor=1,
    gate_type=GateTypes.PLUNGER,
    qick_gen=7,
    slow_dac_channel=3,
    crosscoupling=None,
    max_v=2,
)

gate_d = hardware_config_models.FastGate(
    dc_conversion_factor=4,
    slow_dac_address="slow_dac_address",
    dac_conversion_factor=1,
    gate_type=GateTypes.EXCHANGE,
    qick_gen=8,
    slow_dac_channel=4,
    crosscoupling=None,
    max_v=2,
)
gate_e = hardware_config_models.FastGate(
    dc_conversion_factor=4,
    slow_dac_address="slow_dac_address",
    dac_conversion_factor=1,
    gate_type=GateTypes.PLUNGER,
    qick_gen=9,
    slow_dac_channel=5,
    crosscoupling=None,
    max_v=2,
)
# SlowGate models are used for gates that aren't hooked up to qick, and only require a DC source
gate_f = hardware_config_models.SlowGate(
    dc_conversion_factor=4,
    slow_dac_address="slow_dac_address",
    gate_type=GateTypes.AUX,
    slow_dac_channel=6,
    crosscoupling=None,
    max_v=2,
)

z1 = hardware_config_models.SlowGate(
    dc_conversion_factor=4,
    slow_dac_address="slow_dac_address",
    gate_type=GateTypes.AUX,
    slow_dac_channel=7,
    crosscoupling=None,
    max_v=2,
)

z2 = hardware_config_models.SlowGate(
    dc_conversion_factor=4,
    slow_dac_address="slow_dac_address",
    gate_type=GateTypes.AUX,
    slow_dac_channel=8,
    crosscoupling=None,
    max_v=2,
)

hardware = hardware_config_models.HardwareConfig(
    sd_in=readout_in,
    m1_readout=[readout_out_1],
    m2_readout=[readout_out_2],
    channels={
        "P1": gate_a,
        "X1": gate_b,
        "P2": gate_c,
        "X2": gate_d,
        "P3": gate_e,
        "M1": gate_f,
        "Z1": z1,
        "Z2": z2,
    },
    voltage_source="test",
    rf_gen=1,
)

file_manager.save_config_json(
    hardware, hw_file
)  ### save this file to the hardware config location you listed in settings.py
# print the object we just made as a dict
hw_cfg = hardware.model_dump()
hw_cfg

## Dot Experiment Config
After establishing the physical connections of our hardware to our gates with the `hardware_config`, we still need to keep track of experimental parameters which are shared between different experiments using the same measurement setup/device topology.  We do this with the experiment config.  This PyDantic model holds information about the readout parameters (readout frequency, readout pulse gain, etc), spin-to-charge conversion (spam sequence points and durations) and qubits (i.e. exchange axes, calibration, pulse and idle times).

In [ ]:
### for the most basic config ###
dcs_example = dcs_model.DcsConfigParams(
    adc_trig_offset=0.0,
    dds_freq=1,
    readout_freq=1,
    length=10,
    readout_length=10,
    pulse_gain_readout=0.9,
    slack_delay=0.0,
)
basic_config = full_experiment_model.ExperimentConfig(
    m1_readout=dcs_example, m2_readout=dcs_example, qubit_configs=None
)

file_manager.save_config_json(basic_config, exp_file)

In [ ]:
### for config including pauli spin blockade ###
point = spam_models.SpamStep(
    duration=10,
    gate_list={
        "P1": spam_models.SpamPulse(voltage=0.1),
        "P2": spam_models.SpamPulse(voltage=0.1),
    },
)
ramp = spam_models.SpamStep(
    duration=0.1,
    gate_list={
        "P1": spam_models.SpamRamp(voltage=0.1, voltage_2=0.2),
        "P2": spam_models.SpamPulse(voltage=0.1, voltage_2=0.2),
    },
)
psb_cfg = spam_models.DefaultSpam(
    flush=point, entry_20=ramp, meas=point, entry_11=ramp, exit_11=ramp, idle=point
)
ro_cfg = full_experiment_model.ReadoutParams(
    psb_cfg=psb_cfg, measure_dot="M1", reference=True, thresh=True, threshold=10
)
q1 = full_experiment_model.QubitParams(ro_cfg=ro_cfg, qubit_params=None)
full = full_experiment_model.ExperimentConfig(
    m1_readout=dcs_example, m2_readout=dcs_example, qubit_configs={"q1": q1}
)
file_manager.save_config_json(full, exp_file)

In [ ]:
### for full exchange-only qubit config ###
n_gates = qubit_models.ExchangeGateMapParams(
    px=qubit_models.ExchangeGateParams(
        name="P2",
        gate_voltages=qubit_models.ExchangeVoltages(idle_voltage=0, exchange_voltage=0),
    ),
    py=qubit_models.ExchangeGateParams(
        name="P3",
        gate_voltages=qubit_models.ExchangeVoltages(idle_voltage=0, exchange_voltage=0),
    ),
    x=qubit_models.ExchangeGateParams(
        name="X2",
        gate_voltages=qubit_models.ExchangeVoltages(
            idle_voltage=0, exchange_voltage=0.1
        ),
    ),
)
n = qubit_models.ExchangeAxisConfig(
    gates=n_gates, times=qubit_models.ExchangeTimes(idle_time=0.02, exchange_time=0.01)
)
z_gates = qubit_models.ExchangeGateMapParams(
    px=qubit_models.ExchangeGateParams(
        name="P1",
        gate_voltages=qubit_models.ExchangeVoltages(idle_voltage=0, exchange_voltage=0),
    ),
    py=qubit_models.ExchangeGateParams(
        name="P2",
        gate_voltages=qubit_models.ExchangeVoltages(idle_voltage=0, exchange_voltage=0),
    ),
    x=qubit_models.ExchangeGateParams(
        name="X1",
        gate_voltages=qubit_models.ExchangeVoltages(
            idle_voltage=0, exchange_voltage=0.1
        ),
    ),
)
z = qubit_models.ExchangeAxisConfig(
    gates=z_gates, times=qubit_models.ExchangeTimes(idle_time=0.02, exchange_time=0.01)
)
qubit = qubit_models.Eo1QubitAxes(z=z, n=n)
q1 = full_experiment_model.QubitParams(ro_cfg=ro_cfg, qubit_params=qubit)
full = full_experiment_model.ExperimentConfig(
    m1_readout=dcs_example, m2_readout=dcs_example, qubit_configs={"q1": q1}
)

file_manager.save_config_json(full, exp_file)

## DotExperiment
The experiment classes located in `spinqick.experiments` are user friendly wrappers for code written in the QICK API which include data saving and plotting functionality. Each of the classes inherits the DotExperiment class from `spinqick.core`. This class contains the `hardware_config` and `dot_experiment_config` as attributes, and has machinery to convert these config models from native RFSoC units to the physical units uses to define real experiment parameters (volts, microseconds, etc) in the `full_config.json` and `hardware_config.json` files. It also contains the `@dot_experiment.updater` decorator which is used to automatically load all parameters from the full_config file before an experiment is run. This ensures that, if a readout parameter is changed in the config file between two experimental calls across different cells, notebooks, or Python kernels, all information is up to date.

## DC Voltage source

While fast control and readout is executed on the RFSoC, DC bias voltages are provided via an external precision DC source. SpinQICK is setup to be flexible as to which DC source you choose since these sources vary from lab-to-lab. The spinQICK experiments all take a DC source object as an arguement, so one can save the DC voltages on all gates every time you save data. For testing of the rfsoc alone, we will use a dummy voltage source object for now. See notebook `01_dc_voltage_source_setup.ipynb` for more information on setting up a dc source to work with spinQICK.

In [ ]:
dummy_vsource = hardware_manager.DummyDCSource()

## Running an experiment

Now we're ready to try running an experiment.  We will load a simple experiment from the `system_calibrations` module. This experiment just sweeps a single parameter called `adc_trig_offset` which is the delay between when the dac outputs a pulse and when the adc begins to collect data.

In [ ]:
# initialize experiment class from spinqick.experiments
cal_exp = system_calibrations.SystemCalibrations(
    soccfg, soc, voltage_source=dummy_vsource
)
# run experiment, times in units of microseconds
sqickdata = cal_exp.sweep_adc_trig_offset(
    times=(0, 10, 10), point_avgs=10, full_avgs=10, loop_slack=0.1
)

### Experiment output

Most experiments in the `spinqick.experiments` output an object from the `spinqick.core.spinqick_data` module.  This object contains data generated by the experiment, and a metadata about the experiment as well. It also contains a convenient method for saving data and metadata. We will explore this further in a future notebook, but for now here is an example of how to grab the data that was plotted above.

In [ ]:
### analyzed data
ana = sqickdata.analyzed_data[0][0]
x = sqickdata.axes["time"]["sweeps"]["adc_trig_delay"]["data"]
plt.figure()
plt.plot(x, ana)
plt.xlabel("adc trigger delay (us)")
plt.ylabel("dcs conductance (adc units)")